In [1]:
import json

ihops = [json.loads(line) for line in open('ihop_list.jl')]

In [2]:
from ipyleaflet import Map, Marker, MarkerCluster
from IPython.core.display import display

ihop_map = Map(center=(37.8, -96), zoom=4)

marker_cluster = MarkerCluster(
    markers=list(map(lambda ihop: Marker(
        location=(float(ihop['latitude']),float(ihop['longitude'])),
        title=ihop['address'],
        draggable=False,
    ), ihops))
)
ihop_map.add_layer(marker_cluster)

display(ihop_map)

Map(center=[37.8, -96], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [3]:
from ipyleaflet import Map, CircleMarker, MarkerCluster
from IPython.core.display import display

ihop_map = Map(center=(37.8, -96), zoom=4)

for ihop in ihops:
    ihop_map.add_layer(CircleMarker(
        location=(float(ihop['latitude']),float(ihop['longitude'])),
        title=ihop['address'],
        draggable=False,
        radius=1,
    ))

display(ihop_map)

Map(center=[37.8, -96], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…